## 모델 저장과 복원

* 모델을 훈련시키고 다시 쓸 수 있도록 모델 파라미터를 디스크에 저장
* 훈련하는 동안 일정한 간격으로 체크포인트를 저장하여 훈련 중간에 문제가 일어나도 마지막 체크포인트 부터 진행 가능

In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [5]:
housing = fetch_california_housing()

m, n = housing.data.shape
print(m, n)

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]   # (m, 1)행렬을 만든뒤 housing데이터의 값을 채워넣음.
scaled_housing_data_plus_bias

20640 8


array([[ 1.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [ 1.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 1.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 1.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [8]:
n_epochs = 1000
learning_rate = 0.01

# 이전 사용 코드
# X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")             # (20640, 9)
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')      # (20640, 1)

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')     # (9, 1)
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)  # theta 라는 변수에 아래 연산을 할당..

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)          # 변수 초기화
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
        
        sess.run(training_op, feed_dict={X: scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)})
        
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')

학습된 모델을 복원할 때는 init 노드를 사용하여 변수를 초기화하는 대신 Saver객체의 restore를 이용하여 메소드를 호출

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")       # 학습 모델 복원
    best_theta_restored = theta.eval() # 책에는 없습니다.

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [13]:
np.allclose(best_theta, best_theta_restored)

True

Saver는 theta 변수만 weights란 이름으로 저장

In [14]:
saver = tf.train.Saver({"weights": theta})

save() 메소드는 기본적으로 .meta확장자를 가진 동일 이름의 두번째 파일에 그래프 구조를 저장한다.<br/>
tf.train.import_meta_graph()를 사용해 이 그래프 구조를 읽어 드림

In [19]:
tf.reset_default_graph()        # 기본 그래프 초기화

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")    # 그래프 구조 로드
theta = tf.get_default_graph().get_tensor_by_name("theta:0")           # weights 로드

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")        # 학습 모델 복원
    best_theta_restored2 = theta.eval() # 책에는 없습니다.

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [16]:
np.allclose(best_theta, best_theta_restored2)

True